In [0]:
import json
from pyspark.sql import Row

In [0]:
catalog='operations'
schema= 'finance'
table_name = 'dim_tag'

In [0]:
with open("json/dim_tag.json", "r") as file:
    qry = json.load(file)

In [0]:
rows = []
for main_category, subcats in qry.items():            
    for subcat, items in subcats.items():           
        for item in items:                            
            rows.append(Row(main_category=main_category, sub_category=subcat, item=item))

spark.createDataFrame(rows).createOrReplaceTempView('tags_df')

In [0]:
tags_df = spark.sql(f"""
select   sha2(concat_ws('|', item), 256) AS tag_key_hash
,bigint(substr(xxhash64(concat_ws('|', item)), 1, 18)) AS tag_bigint_key
,* from tags_df""")

In [0]:
(
    tags_df
    .write
    .format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "true")
    .saveAsTable(f"{catalog}.{schema}.{table_name}")
)